In [4]:
!git clone https://github.com/sismetanin/rureviews

fatal: destination path 'rureviews' already exists and is not an empty directory.


In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
df = pd.read_csv("/content/rureviews/women-clothing-accessories.3-class.balanced.csv", sep = "\t")

FileNotFoundError: [Errno 2] No such file or directory: '/content/rureviews/women-clothing-accessories.3-class.balanced.csv'

In [4]:
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [5]:
df['sentiment'].value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [6]:
df = df[df['sentiment']!= 'neautral']

In [7]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 8.2 MB 73.2 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=d4d85ca012524d983b2042d8a6eba69985ebc3bd282a60b105b33ec2ce513974
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [8]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import re
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
df['review'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
morph = pymorphy2.MorphAnalyzer()

In [12]:
morph.parse("сделали")

[Parse(word='сделали', tag=OpencorporaTag('VERB,perf,tran plur,past,indc'), normal_form='сделать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'сделали', 648, 4),))]

In [13]:
morph.parse("сделали")[0].normal_form

'сделать'

In [14]:
df['review'] = df['review'].progress_apply(lambda x: word_tokenize(x))

100%|██████████| 60000/60000 [00:09<00:00, 6426.14it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df['review']

0        [качество, плохое, пошив, ужасный, горловина, ...
1        [Товар, отдали, другому, человеку, я, не, полу...
2        [Ужасная, синтетика, Тонкая, ничего, общего, с...
3        [товар, не, пришел, продавец, продлил, защиту,...
4        [Кофточка, голая, синтетика, носить, не, возмо...
                               ...                        
89995    [сделано, достаточно, хорошо, на, ткани, сдела...
89996    [Накидка, шикарная, Спасибо, большое, провдо, ...
89997    [спасибо, большое, продовца, рекомендую, заказ...
89998    [Очень, довольна, заказом, Меньше, месяца, в, ...
89999    [хорошая, куртка, постороннего, запаха, нет, ш...
Name: review, Length: 60000, dtype: object

In [16]:
df['review_lemmatized']= df['review'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])

100%|██████████| 60000/60000 [03:52<00:00, 258.59it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 33.6 MB/s 
     |████████████████████████████████| 133 kB 30.0 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=1b7e1e9a6238eb24a9a0ab6aabcb30fbdc1eae8f06c73390800f72849239a222
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import eli5

In [19]:
vectorizer = TfidfVectorizer(ngram_range = (1,2))
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))

In [20]:
X.shape

(60000, 396100)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size = 0.33, random_state = 42)

In [45]:
logit = LogisticRegression(random_state = 17, multi_class='multinomial') #remove multinomial

In [46]:
logit.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(multi_class='multinomial', random_state=17)

In [47]:
y_pred = logit.predict_proba(X_test)

In [48]:
accuracy_score(y_test, y_pred)

ValueError: ignored

In [57]:
accuracy_score(y_test.sample(frac=1), y_pred)

ValueError: ignored

In [49]:
roc_auc_score(y_test, y_pred[:, 1])

0.979634242200931

In [50]:
eli5.show_weights(estimator=logit, feature_names = list(vectorizer.get_feature_names()), top=(50,50))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+6.140,отличный
+5.814,хороший
+4.632,супер
+4.403,немного
+4.025,отлично
+3.932,хорошо
+3.911,спасибо
+3.666,приятный
+3.654,классный
+3.506,понравиться


In [51]:
from gensim.models import Phrases
documents = ["Ландыши их подарил мне ты", "Цветы в вазе", "Ландыши весенние цветы"]

sentence_stream = [doc.split(" ") for doc in documents]
bigram = Phrases(sentence_stream, min_count=1, threshold=2)
sent = ['Он', 'поставил', 'Ландыши', 'в','вазу']
print(bigram[sent])

['Он', 'поставил', 'Ландыши', 'в', 'вазу']


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [52]:
[doc.split(" ") for doc in documents]

[['Ландыши', 'их', 'подарил', 'мне', 'ты'],
 ['Цветы', 'в', 'вазе'],
 ['Ландыши', 'весенние', 'цветы']]

In [63]:
!pip3 install --upgrade -r requirements.txt -e .

ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content


In [2]:
!python3 -m pip install skopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt


In [1]:
!pip install scikit-learn==0.24.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install skopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt


In [54]:
import skopt
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
  SVC(),
    {
         'C': Real(1e-6, 1e+6, prior='log-uniform'),
         'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
         'degree': Integer(1,8),
         'kernel': Categorical(['linear', 'poly', 'rbf']),
     },
     n_iter=32,
     random_state=0
 )

# executes bayesian optimization
_ = opt.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(opt.score(X_test, y_test))

ModuleNotFoundError: ignored